In [1]:
import torch
import torch.nn as nn
from nltk.tokenize import word_tokenize
from os import path
from math import sqrt

In [2]:
class Vocabulary:
	def __init__(self):
		self.vocabulary = set('')
		self.stoi = {'':0}
		self.itos = {0:''}

	def add(self, v):
		if type(v) == str:
			self.vocabulary(v)
		elif type(v) == list:
			self.vocabulary = self.vocabulary.union(set(v)) 

	def create_mappings(self):
		self.stoi = {v:i+1 for i, v in enumerate(self.vocabulary)}
		self.itos = {i+1:v for i, v in enumerate(self.vocabulary)}

	def encode(self, s): 
		return [self.stoi[c] for c in s]
	
	def decode(self, i): 
		return [self.itos[n] for n in i]
	

class PreProcessor:
	def __init__(self):
		self.english_vocabulary = Vocabulary()
		self.cherokee_vocabulary = Vocabulary()
		self.cherokee = []
		self.english = []
		self.max_length = 0
		self.count = 0

	def load_text(self, file_name):
		data, language = [], file_name.split('.')[1]

		with open(file_name) as f:
			for line in f.readlines():
				sentence = word_tokenize(line)

				if language == 'en': 
					self.english_vocabulary.add(sentence)
				else:
					self.cherokee_vocabulary.add(sentence)

				self.max_length = max(self.max_length, len(sentence))
				data.append(sentence)
				self.count += 1 
		return data
	
	def get_data(self, file_set):
		cherokee = self.load_text(path.join('chr_en_data', f'{file_set}.chr'))
		english  = self.load_text(path.join('chr_en_data', f'{file_set}.en' ))
		assert len(cherokee) == len(english)
		self.cherokee += cherokee
		self.english  += english

		return cherokee, english

	
	def create_tensors(self):
		self.english_vocabulary.create_mappings()
		self.cherokee_vocabulary.create_mappings()

		english  = torch.zeros(size=(self.count//2, self.max_length), dtype=int)
		cherokee = torch.zeros(size=(self.count//2, self.max_length), dtype=int)

		for i, sen in enumerate(self.english):
			for j, v in enumerate(self.english_vocabulary.encode(sen)):
				english[i, j] = v
		
		for i, sen in enumerate(self.cherokee):
			for j, v in enumerate(self.cherokee_vocabulary.encode(sen)):
				cherokee[i, j] = v

		self.cherokee, self.english = cherokee, english


preprocessor = PreProcessor()
preprocessor.get_data('dev')
preprocessor.get_data('test')
preprocessor.get_data('train')
preprocessor.create_tensors()

test = word_tokenize('ᏣᏌᏙᏰᏃ ᎢᎦᎦᏛ ᏓᏳᏂᎷᏤᎵ ᏂᎬᎾᏛ ᏗᏁᎯ.')

assert preprocessor.cherokee_vocabulary.decode(preprocessor.cherokee_vocabulary.encode(test)) == test
assert preprocessor.english.shape == preprocessor.cherokee.shape

In [87]:
cherokee_vocab_size, english_vocab_size = len(preprocessor.cherokee_vocabulary.vocabulary), len(preprocessor.english_vocabulary.vocabulary)

print(f'Cherokee Vocabulary Size: {cherokee_vocab_size}')
print(f'English  Vocabulary Size: {english_vocab_size}')

Cherokee Vocabulary Size: 44564
English  Vocabulary Size: 13928


In [81]:
EMBEDDING_DIMENSIONS = 2
SEQUENCE_LENGTH      = 5
QKV_DIMENSIONS       = 2
ATTENTION_HEADS      = 2

In [77]:
class AttentionHead(nn.Module):
	def __init__(self):
		super().__init__()
		self.obtain_key   = nn.Linear(EMBEDDING_DIMENSIONS, QKV_DIMENSIONS)
		self.obtain_query = nn.Linear(EMBEDDING_DIMENSIONS, QKV_DIMENSIONS)
		self.obtain_value = nn.Linear(EMBEDDING_DIMENSIONS, QKV_DIMENSIONS)

	def forward(self, data):
		Q, K, V = self.obtain_key(data), self.obtain_query(data), self.obtain_value(data)
		mat_mul = Q @ K.transpose(-2, -1)
		scaled_mat_mul = mat_mul / sqrt(QKV_DIMENSIONS)
		softmax_mat_mul = torch.softmax(scaled_mat_mul, dim=-1)
		output = softmax_mat_mul @ V

		return output



In [86]:
class MultiHeadedAttention(nn.Module):
	def __init__(self):
		super().__init__()
		self.heads  = [AttentionHead() for _ in range(ATTENTION_HEADS)]
		self.linear = nn.Linear(EMBEDDING_DIMENSIONS*ATTENTION_HEADS, EMBEDDING_DIMENSIONS)

	def forward(self, data):
		vectors = torch.cat([head(data) for head in self.heads], dim=-1)
		return self.linear(vectors)
	

test = torch.randn(size=(SEQUENCE_LENGTH, EMBEDDING_DIMENSIONS))
test_module = MultiHeadedAttention()
print(test_module(test))


tensor([[-0.4945, -0.9357],
        [-0.5232, -0.9228],
        [-0.6216, -0.8806],
        [-0.7726, -0.8045],
        [-0.5299, -0.9202]], grad_fn=<AddmmBackward0>)


In [95]:
class FeedForward(nn.Module):
	def __init__(self):
		super().__init__()
		self.network = nn.Sequential(
			nn.Linear(EMBEDDING_DIMENSIONS, EMBEDDING_DIMENSIONS),
			nn.ReLU(),
			nn.Linear(EMBEDDING_DIMENSIONS, EMBEDDING_DIMENSIONS)
		)

	def forward(self, data):
		return self.network(data)

test = torch.randn(size=(SEQUENCE_LENGTH, EMBEDDING_DIMENSIONS))
test_module = FeedForward()
print(test_module(test))


tensor([[ 0.1438, -0.2841],
        [ 0.1535, -0.5275],
        [ 0.1464, -0.5014],
        [ 0.1329, -0.0187],
        [ 0.1472, -0.5060]], grad_fn=<AddmmBackward0>)


In [149]:
class Encoder(nn.Module):
	def __init__(self):
		super().__init__()
		self.word_embeddings = nn.Embedding(cherokee_vocab_size, EMBEDDING_DIMENSIONS)
		self.positional_encodings = nn.Embedding(SEQUENCE_LENGTH, EMBEDDING_DIMENSIONS)
		self.multi_headed_attention = MultiHeadedAttention()
		self.norm1 = nn.LayerNorm(EMBEDDING_DIMENSIONS)
		self.feed_forward = FeedForward()
		self.norm2 = nn.LayerNorm(EMBEDDING_DIMENSIONS)

	def forward(self, data):
		encodings = self.word_embeddings(data) + self.positional_encodings(torch.tensor([i for i in range(SEQUENCE_LENGTH)]))
		attention_vectors = self.multi_headed_attention(encodings)
		normalised = self.norm1(attention_vectors) + encodings # Residual Connection
		fed_through = self.feed_forward(normalised)
		normalised_2 = self.norm2(fed_through) + normalised

		return normalised_2
	
test = torch.randint(low=0, high=cherokee_vocab_size-1, size=(SEQUENCE_LENGTH,))
test_module = Encoder()
print(test_module(test))

tensor([[-3.1532,  1.9609],
        [-3.6078,  2.0312],
        [-2.2964,  3.5502],
        [-5.1408,  1.4809],
        [-0.9624, -2.0376]], grad_fn=<AddBackward0>)
